In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [54]:
htmls = []
flag = False
for i in range(1,8):
    url = "http://www.dosw.gov.taipei/lp.asp?CtNode=22120&CtUnit=10403&BaseDSD=69&mp=107001&nowPage="+ str(i) +"&pagesize=100"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    links = soup.select("#zone.content11 .list")[0].find_all('a')[1:]
    for link in links:
        html_dict = {}
        url = "http://www.dosw.gov.taipei/" + link['href']
        if  url == "http://www.dosw.gov.taipei/ct.asp?xItem=180697299&ctNode=22120&mp=107001":
            flag = True
        if flag:
            print(url)
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'lxml')

            h3 = soup.find_all('h3')[0]
            question = h3.text
            ans = h3.find_next('p').text
            all_ems = h3.find_next('ul').find_all('em')
            
            cat = [em.text for em in all_ems if '臺北市' in em.text]
            if len(cat) > 0:
                cat = cat[0] 
                print(cat)
            else:
                cat = None
                print('no cat')

            html_dict['question'] = question
            html_dict['ans'] = ans
            html_dict['cat'] = cat
            print(html_dict)
            htmls.append(html_dict)

http://www.dosw.gov.taipei/ct.asp?xItem=180697299&ctNode=22120&mp=107001
no cat
{'cat': None, 'question': '臺北市立陽明教養院目前安置照顧情形', 'ans': ''}
http://www.dosw.gov.taipei/ct.asp?xItem=157314830&ctNode=22120&mp=107001
臺北市政府社會局身心障礙者福利科
{'cat': '臺北市政府社會局身心障礙者福利科', 'question': '生活輔具 (器具補助、輔具服務)申請資格', 'ans': '身心障礙者輔具費用補助新制自101年7月11日起施行，申請資格如下：\r\n 1.設籍臺北市。\r\n 2.領有臺北市核（換、補）發或註記之身心障礙手冊或證明；最近一年居住國內達183日。\r\n 3.申請補助項目係未獲政府其它醫療補助、社會保險給付或相同性質（輔助器具）之補助者。\r\n 4.已超過輔助器具之補助年限。\r\n 5.補助內容：詳見內政部「身心障礙者輔具費用補助辦法」、「身心障礙者輔具費用補助基準表」及其他相關規定。（可向區公所社會課索取或至臺北市政府社會局網站www.dosw.taipei.gov.tw下載）'}
http://www.dosw.gov.taipei/ct.asp?xItem=159017065&ctNode=22120&mp=107001
臺北市政府社會局身心障礙者福利科
{'cat': '臺北市政府社會局身心障礙者福利科', 'question': '身心障礙者人工電子耳應備文件', 'ans': '1.申請書正本。\r\n2.最近3個月內由醫學中心或衛生福利部專案核可醫院之耳鼻喉科醫生開立之診斷證明書正本1份。(應詳述優、劣耳聽力損害程度:dBHL數)\r\n3.術前狀況評估及術後復健計畫書。\r\n(1) 術前狀況評估：由醫院耳鼻喉科醫師、聽力檢查師或語言治療師或職能 治療師或心智科醫師或社工師等專業人士開立，內容應載明病患術前狀況，含聽能復健效果及聽力障礙病史及病患耳蝸之完整性與否，及人工電子耳對其確有實際預期成效及裝置需要，或家庭動力與資源或個案心智狀況等；符合補助標準對象資格已施行植入手術者，請另載明手術施行日期。\r\n(2)術

In [55]:
from pymongo import MongoClient
conn = MongoClient()
db = conn.QA1999
collection = db.taipei

collection.insert_many(htmls)
